# 任务1：命名实体识别任务

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json

# 加载模型
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model_path = './model/'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype="auto", device_map="auto")
print("模型加载完成！")

# 标签映射
label_label_id_mapping = {
    "O": 0,
    "B-PER": 1,
    "I-PER": 2,
    "E-PER": 3,
    "S-PER": 4,
    "B-LOC": 5,
    "I-LOC": 6,
    "E-LOC": 7,
    "S-LOC": 8,
    "B-OFI": 9,
    "I-OFI": 10,
    "E-OFI": 11,
    "S-OFI": 12,
    "B-BOOK": 13,
    "I-BOOK": 14,
    "E-BOOK": 15,
    "S-BOOK": 16,
}

# 读取 CHisIEC 数据
def load_ner_data(file_path):
    print(f"正在从 {file_path} 读取 NER 数据...")
    texts = []
    labels = []
    with open(file_path, "r") as f:
        text = []
        label = []
        for line in f:
            if line.strip() == "":
                if text and label:
                    texts.append("".join(text))
                    labels.append([label_label_id_mapping[l] for l in label])
                text = []
                label = []
            else:
                word, tag = line.strip().split()
                text.append(word)
                label.append(tag)
        if text and label:
            texts.append("".join(text))
            labels.append([label_label_id_mapping[l] for l in label])
    passage = ''.join(texts)
    passage_labels = [label_label_id_mapping[l] for l in label]
    print("NER 数据加载完成！")
    return texts, labels, passage, passage_labels

# 定义 NER prompt 模板
def generate_ner_prompt(text):
    # prompt = f"""请对以下古文句子进行命名实体识别，识别出句子中每个字是否是人物、地点、官职和书籍名称，并对每个字标注标签，格式为“字 标签“，字与字之间用__split__隔开。
    # 标签说明：
    # - B 表示实体的开始，I 表示实体的中间，E 表示实体的结束，S 表示单个字的实体，O 表示非实体；
    # - 实体需要分类：PER 表示人物，LOC 表示地点，OFI 表示官职，BOOK 表示书籍；
    # - 示例标签包括 B-PER、I-LOC、E-OFI、S-BOOK 等。
    # 输出形如：子 B-PER__split__智 E-PER__split__袭 O__split__职 O__split__，O__split__授 O__split__三 B-BOOK__split__珠 I-BOOK__split__虎 I-BOOK__split__符 E-BOOK
    # 需要你分析的句子为：{text}
    # """
    prompt = f"""请对以下古文句子进行命名实体识别，标注每个字是否是人物(PER)、地点(LOC)、官职(OFI)、书籍名称(BOOK)。格式如下：
"字 标签"（字与字之间用__split__隔开）。
标签示例：PER、LOC、OFI、BOOK、O。
示例输出：子智 PER__split__袭 O__split__职 O__split__，O__split__授 O__split__三珠虎符 BOOK。
古文句子：{text}
    """
    return prompt

# 模型推理 - NER
def ner_with_llm(texts: list, ground_truth: list):
    print("开始命名实体识别推理...")
    entities = []
    for i, text in enumerate(texts):
        prompt = generate_ner_prompt(text)
        print(f"Prompt for sentence {i+1}: {prompt}")

        messages = [
            {"role": "system", "content": "你是一个命名实体识别助手，专注于识别给出的句子中的人物、地点、官职和书籍名称等命名实体。"},
            {"role": "user", "content": prompt}
        ]
        
        text_input = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = tokenizer([text_input], return_tensors="pt").to(model.device)

        # 获取模型回答
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=100
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        print(f"Response: {response}")
        entities.append({
            'sentence': text, 
            'reply': response, 
            'gt': ground_truth[i]
        })
    print("所有文本的命名实体识别推理完成！")
    return entities


ner_texts, true_labels, passage, passage_labels = load_ner_data("./data/ner.txt")
print(f"passage: {passage}\nlength of texts: {len(ner_texts)}")
predicted_entities = ner_with_llm(ner_texts, true_labels)

with open('./ner_results.json', 'w', encoding='utf-8') as f:
    json.dump(predicted_entities, f, ensure_ascii=False, indent=4)


模型加载完成！
正在从 ./data/ner.txt 读取 NER 数据...
NER 数据加载完成！
passage: 或以问至德，荅曰：「夫庆赏刑罪，人主之权柄，凡为人臣，岂得与人主争权柄哉！」其慎密如此。后高宗知而深歎美之。仪凤四年薨，辍朝三日，使百官以次赴宅哭之，赠开府仪同三司、并州大都督，谥曰恭。宣帝即位，授上柱国。运之为宫正也，数进谏于帝。帝不纳，反疏忌之。时运又与王轨、宇文孝伯等皆为武帝亲待。阿剌怗木儿袭职，授虎符，緫管高丽人户。至元八年，将兵讨叛贼金通精，贼败走耽罗。十一年，进昭勇大将军，从都元帅忽都征日本国，预有战功。孝武西迁，进车骑大将军、仪同三司，别封万年县伯，乃除华州刺史。齐神武率军进潼关，人怀危惧，罴劝励将士，众心乃安。子智袭职，授三珠虎符、宣武将军，为万户。延祐二年，进明威将军，以病去职。子安世袭。白再荣，本蕃部人也。太宗称善，勑自今后中书门下及三品以上入合，必遣谏官随之。珪每推诚纳忠，多所献替，太宗顾待益厚，赐爵永宁县男，迁黄门侍郎，兼太子右庶子。帝命阇里毕与皇太弟国王分拨诸侯王城邑，谕阇里毕曰：「汉人中若王宣抚者，可任使之。」遂以前职，兼判三司副使。后又命省臣緫括归附工匠之数，将俾大臣分掌之。陈太建时，应召来憩于至真观。期月，又辞入天台山，因绝谷养性，所资唯松水而已，虽隆冬沍寒，不服绵絮。太傅徐陵为之刊山立颂。九月，潞州节度使康君立以酖死。韶以周人反覆，本无信义，比晋阳之役，其事可知。护外讬为相，其实王也。既为母请和，不遣一介之使申其情理，乃据移书，即送其母，恐示之弱。如臣管见，且外许之，待后放之未晚。不听。遂遣使以礼将送。寻遭母忧。时天下丧乱，皆不能终三年丧，唯奂及吴国张种在寇乱中，守法度，并以孝闻。天复二年二月，李嗣昭、周德威领大军自慈、隰进攻晋、绛，营于蒲县。乙未，汴将朱友寕、氏叔琮将兵十万，营于蒲县之南。乙巳，汴帅自领军至晋州，德威之军大恐。三月丁巳，有虹贯德威之营。子伯昱。弟朏，武定中，开府中兵参军。永泰元年，吐蕃请和，诏宰相元载、杜鸿渐与虏使者同盟。至太祖生，眇一目，长而骁勇，善骑射，所向无敌，时谓之「独眼龙」，大为部落所疾。太祖恐祸及，遂举族归唐，授云州刺史，赐姓李，名克用。大和七年，罢邠宁行军司马，入京师，御史李款合内弹之曰「郑注内通勑使，外结朝官，两地往来，卜射财货，昼伏夜动，干窃化权。人不敢言，道路以目。请付法司。

In [ ]:
from sklearn.metrics import f1_score
import json
import re


# 计算 F1-macro 分数
def evaluate_ner_f1(true_labels, predicted_labels):
    print("正在计算 F1-macro 分数...")
    
    # 将嵌套的 List[List[int]] 展平为 List[int] 格式
    true_labels_flat = [label for sublist in true_labels for label in sublist]
    predicted_labels_flat = [label for sublist in predicted_labels for label in sublist]
    print(f"gt:{true_labels_flat}\nprediction: {predicted_labels_flat}")
    
    # 计算 F1-macro 分数
    score = f1_score(true_labels_flat, predicted_labels_flat, average="macro")
    print("F1-macro 分数计算完成！")
    return score


def consolidate_entities(sentence, tags):
    """将连续的相同标签整理成词，并标注类别"""
    words = []
    current_word = []
    current_tag = "O"

    for tag in tags:
        # 过滤乱码，仅保留符合格式的标签
        if not re.match(r"^[\u4e00-\u9fa5a-zA-Z]+ [A-Z]+$", tag):
            print(f"Skipping invalid tag: {tag}")  # 打印日志
            continue
        
        char, entity = tag.split()
        if entity != current_tag or (current_word and sentence[len(current_word) - 1] != char):
            if current_word:
                words.append(f"{''.join(current_word)} {current_tag}")
            current_word = [char]
            current_tag = entity
        else:
            current_word.append(char)

    if current_word:
        words.append(f"{''.join(current_word)} {current_tag}")

    return words


def process_entities(words, label_id_mapping, sentence):
    """根据整理后的词列表生成 B-I-E 格式的标签"""
    tags = [label_id_mapping["O"]] * len(sentence)

    for word in words:
        word_content, tag = word.rsplit(" ", 1)
        start_idx = sentence.find(word_content)

        if tag == "O" or start_idx == -1:
            continue

        n = len(word_content)
        for i in range(n):
            if tag in ['BOOK', 'PER', 'LOC', 'OFI']:
                if i == 0:
                    tags[start_idx + i] = label_id_mapping[f"B-{tag}"]
                elif i == n - 1:
                    tags[start_idx + i] = label_id_mapping[f"E-{tag}"]
                else:
                    tags[start_idx + i] = label_id_mapping[f"I-{tag}"]
            else:
                tag = 'O'

    return tags


def process_sentence(predicted_entity: dict, label_id_mapping: dict):
    """逐句处理"""
    true_labels = predicted_entity['gt']  # Ground truth 标签
    sentence = predicted_entity['sentence']  # 句子内容
    reply = predicted_entity['reply']  # 模型回复
    tags = reply.split('__split__')

    # 整理连续性
    consolidated_words = consolidate_entities(sentence, tags)
    print(f"Consolidated words: {consolidated_words}")

    # 生成 B-I-E 标签
    reply_tags = process_entities(consolidated_words, label_id_mapping, sentence)

    return true_labels, reply_tags


global_true_labels = []
global_predicted_labels = []

with open('./ner_results.json', 'r') as f:
    predicted_entities = json.load(f)

# label 的映射，便于核对
label_id_mapping = {
    'B-PER': 0, 'I-PER': 1, 'E-PER': 2, 'S-PER': 3,
    'B-LOC': 4, 'I-LOC': 5, 'E-LOC': 6, 'S-LOC': 7,
    'B-OFI': 8, 'I-OFI': 9, 'E-OFI': 10, 'S-OFI': 11,
    'B-BOOK': 12, 'I-BOOK': 13, 'E-BOOK': 14, 'S-BOOK': 15,
    'O': 16
}

for predicted_entity in predicted_entities:
    true_labels, predicted_labels = process_sentence(predicted_entity, label_id_mapping)
    global_true_labels.append(true_labels)
    global_predicted_labels.append(predicted_labels)

# 计算整体 F1-macro 分数
f1_macro_score = evaluate_ner_f1(global_true_labels, global_predicted_labels)
print(f"NER 任务的整体 F1-macro 分数: {f1_macro_score}")


Skipping invalid tag: ^{1}^{2}__^{3}__^{4}__^{5}__^{6}__^{7}__^{8}__^{9}__^{10}__^{11}__^{12}__^{13}__^{14}__^{15}__^{16}__^{17}__^{18}__^{19}__^{20}__^{21}
Consolidated words: ['至德 PER']
Skipping invalid tag: ，O
Skipping invalid tag: ，O
Skipping invalid tag: ，O
Skipping invalid tag: ，O
Skipping invalid tag: ，O
Skipping invalid tag: ，O
Skipping invalid tag: 时运又与王轨、宇文孝伯等
Consolidated words: ['宣帝 PER', '即位 O', '授 O', '上柱国 O', '运之为宫正也 O', '数进谏于帝 O', '帝不纳 O', '反疏忌之 O']
Skipping invalid tag: ，O
Skipping invalid tag: 緫管高丽人户. 至元八年，
Skipping invalid tag: 将兵讨叛贼金通精，
Skipping invalid tag: 贼败走耽罗。 十一年，
Skipping invalid tag: 进昭勇大将军，
Skipping invalid tag: 从都元帅忽都征日本国，
Skipping invalid tag: 
Consolidated words: ['阿剌怗木儿 PER', '袭 OFI', '职 O', '授 O', '虎符 O']
Skipping invalid tag: 。
Consolidated words: ['孝武 PER', '西迁 O', '进车骑大将军 O', '仪同三司 O', '别封万年县伯 O', '乃除华州刺史 O', '齐神武 PER', '率军进潼关 O', '人怀危惧 O', '罴劝励将士 O', '众心乃安 O']
Skipping invalid tag: ，O
Skipping invalid tag: 三珠虎符 BOOK。
子安世 PER
Skipping invalid tag: ，

## 结果分析

### 1. prompt尝试

尝试了简洁的prompt，如“请帮我识别以下古文句子中的命名实体，包括人名、官职、地点、书籍”，但是模型会输出很多无关的噪音，难以进行数据处理，且容易生成无关内容。也尝试了比较反复的prompt（如注释）但可能prompt过长妨碍模型理解，导致结果比现在更差。


### 2. 识别的问题与难点

- 模型输出难以保持prompt中提到的规律，若不按规则进行输出，后续处理数据非常麻烦，难有一套标准，导致可能遗漏了很多正确答案（如：“王 H UAN PER”）。
- 让模型学会这个任务是什么非常困难，容易产出奇怪的输出（如：“^{1}^{2}\__^{3}\__^{4}__”）
- 以上两点导致对模型的输出进行数据处理非常困难


## 比较 LLM 和 BiLSTM 在古文命名实体识别中的区别

| 特性                | LLM                                           | BiLSTM                                      |
|---------------------|-----------------------------------------------|--------------------------------------------|
| **模型架构**        | 预训练的多层 Transformer，支持大规模并行计算 | 双向 LSTM 网络，擅长处理序列依赖关系        |
| **输入处理**        | 通过上下文理解，结合 Prompt，生成输出         | 通过词嵌入和序列上下文捕获特定信息          |
| **依赖的资源**      | 大规模预训练数据、GPU/TPU                    | 训练好的词嵌入（如 Word2Vec）和较少计算资源 |
| **对古文的适应性**  | 对古文预训练较少，可能需要特定领域的微调       | 如果词嵌入词表覆盖不足，则需要额外数据扩充   |
| **上下文理解能力**  | 强，能够结合上下文推断                      | 较弱，对长距离上下文依赖的处理有限           |
| **输出的一致性**    | 可能输出冗余或多余标签                       | 依赖于标签分布，输出一致性较好              |
| **训练开销**        | 无需训练（使用现有预训练模型）                | 需要从零训练或微调，耗时耗资源               |
| **推理速度**        | 较慢，受模型大小和硬件限制影响                | 较快，适合批量处理                          |
| **可扩展性**        | 随硬件扩展，但模型复杂度限制                  | 简单可扩展，适合特定任务优化                 |

---

1. LLM 适合处理复杂句法和语义关系，尤其是多层次结构的古文；通过编写 Prompt 即可快速应用，适合少量任务；利用大规模预训练知识，在缺乏训练数据时表现较好。
    然而，Prompt 编写质量直接影响性能，优化成本较高，；LLM 推理需要高性能硬件，对长文本处理可能受限；对于古文等特定领域，可能需要微调以提升性能。

2. BiLSTM 模型较小，计算成本低，适合特定领域任务；可以针对古文构造特定的词嵌入和标签集；适合在任务数据丰富的情况下优化模型。
    然而，BiLSTM 对长距离依赖的理解不如 LLM；且需要大量高质量标注数据训练，且对低资源领域效果较差；如果预训练词嵌入对古文支持不足，需要额外的预处理。


先前我使用 BiLSTM 识别古文实体得到的结果为：`{'accuracy': 0.9237534414193943, 'f1_macro': 0.6876196662034736}`，此次使用 LLM 得到的结果远低于 BiLSTM，原因可能有：
- 任务所需的标签（如 PER、LOC）与 LLM 预训练任务（如语言建模）不直接相关，虽进行了说明，但 LLM 生成的结果中仍噪音较多，难以进行统一的数据处理，导致计算出的结果偏低；
- prompt 仍有待优化；
- LLM 缺乏对特定领域的知识，在古文解析方面表现不佳，而 BiLSTM 是专门针对古文 NER 任务训练的模型；



# 任务2：摘要任务

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json

# 加载模型
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model_path = './model/'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype="auto", device_map="auto")
print("模型加载完成！")


# 定义摘要 prompt 模板
def generate_summary_prompt(text: str) -> str:
    '''
    @param: text 新闻文本
    
    @return: prompt 生成的 prompt
    '''
    prompt = f"请为以下文本生成摘要，不要添加额外解释，仅输出摘要内容。新闻内容如下：{text}"
    return prompt


# 模型推理 - 摘要
def summarization_with_llm(data: list) -> list:
    '''
    @param: data 预测实例，每个元素包含目标新闻文本和标准摘要答案
    
    @return: predictions 预测后的实例，每个元素包含目标新闻文本、标准摘要答案和预测结果
    '''
    predictions = []
    for d in data:
        text = d['article']
        prompt = generate_summary_prompt(text)
        
        # 生成 prompt 并获取模型输出
        messages = [
            {"role": "system", "content": "你是一个新闻摘要生成助手，用于生成给定新闻的摘要。"},
            {"role": "user", "content": prompt}
        ]
        formatted_text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        
        model_inputs = tokenizer([formatted_text], return_tensors="pt").to(model.device)
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=1000 
        )
        summary = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0].replace("你是一个新闻摘要生成助手，用于生成给定新闻的摘要。", '').replace(prompt, '')
        print("ans:", summary)

        predictions.append({
            'article': text, 
            'ans': summary, 
            'ground truth': d['summary']
        })
    return predictions


data = []
with open("./data/summary.jsonl", 'r') as f:
    for line in f:
        data.append(json.loads(line))
predicted_summaries = summarization_with_llm(data)

# 将结果输出到json文件便于后续读取
with open('./abs_results.json', 'w', encoding='utf-8') as f:
    json.dump(predicted_summaries, f, ensure_ascii=False, indent=4)

/home/Data/.bin/.config/dz/anaconda3/envs/py_39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/Data/.bin/.config/dz/anaconda3/envs/py_39/lib/python3.9/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/Data/.bin/.config/dz

模型加载完成！
ans: system

user

assistant
福建省法治报 - 海峡法治在线 6月 1日讯,2013年9月,吴某和罗某等人在连城县莲峰镇某休闲吧包厢喝酒时,服务员因灯开关错误把所有灯关上,导致其中一人受伤并摔碎啤酒瓶。吴某发现后将包厢内桌椅掀翻,与服务员发生理论冲突后拳打脚踢,破坏了包厢财物等。连城县法院近日审理此案,认定吴某构成故意毁坏财物罪,判处有期徒刑六个月,缓刑一年。
ans: system

user

assistant
宫宗良和姜翠卿参加婚礼婚纱照。
ans: system

user

assistant
2月2日晚上11点左右，住在江苏省扬州市石油山庄的市民顾女士听到隔壁车位发出的敲窗户声音，发现车内被困一人和一名老人。
ans: system

user

assistant
7月19日,卧龙区陆营镇某村村民李某因琐事与家人生气独自喝酒,然后拿起菜刀企图自杀,急救人员及时抢救,目前李某已出院回家。
ans: system

user

assistant
华硕 ZenFone 3 . Deluxe 将全球首次搭载骁龙 821 处理器 , 8月份将在台湾首发上市 , 售价 24990 新台币 , 约合 人民币 5200 元 .
ans: system

user

assistant
中国电信董事长常小兵因涉嫌严重违纪被调查。
ans: system

user

assistant
15日夜间至16日白天,南平、宁德和三明北部有中雨或雷阵雨,局地暴雨;其余地区多云到阴,有阵雨或雷阵雨,局部大暴雨;全省大部分地区有中雨或雷阵雨,局部暴雨。
ans: system

user

assistant
网友怀疑司机下身不正常，车主被指责为“猥琐”司机。
ans: system

user

assistant
暴雨黄色预警升级橙色预警
ans: system

user

assistant
余杭辉带着锦旗到公交二公司感谢。暴雨来临时，他决定绕道含香菜场去买菜。由于没有道路可以通行，他选择了沿小溪走。没想到的是，就在这瞬间，情况发生了变化。余辉觉得自己应该过去，但是被洪水冲走了。他的孩子只有15个月大，家人也慌了神。村民们提供救生圈和绳子，帮助他安全地转移到安全地带。当时，他还在游泳，但老婆不会游泳。村民还给了他一些救生圈和衣

In [2]:
import json
from rouge import Rouge
import sys
import jieba  # 导入分词库

# 增加递归深度限制
sys.setrecursionlimit(1500)

# 对文本进行分词
def tokenize_text(text):
    """
    对文本进行分词处理
    @param text: 输入的文本
    @return: 分词后的字符串
    """
    tokens = jieba.lcut(text)  # 使用 jieba 分词
    return " ".join(tokens)  # 分词结果用空格连接

# 计算 ROUGE 分数
def evaluate_summary_rouge(predicted_summaries: list) -> tuple:
    '''
    @param: predicted_summaries 预测后的实例，每个元素包含目标新闻文本、标准摘要答案和预测结果

    @return: avg_rouge_1 ROUGE-1 分数
    @return: avg_rouge_2 ROUGE-2 分数

    @desc: 对每条新闻先计算 ROUGE 分数，最后汇总
    '''
    rouge = Rouge()
    rouge_1_scores, rouge_2_scores = [], []

    for record in predicted_summaries:
        # 对 reply 和 ground truth 进行预处理
        ans = record['ans'].replace("system", "").replace("assistant", "").replace("user", "").replace("\n", "")
        ground_truth = record['ground truth']

        # 分词处理
        ans = tokenize_text(ans)
        ground_truth = tokenize_text(ground_truth)

        # 确保字符串适中（如果必要的话可以截断）
        if len(ans) > 500:
            ans = ans[:500]
        if len(ground_truth) > 500:
            ground_truth = ground_truth[:500]

        # 计算单条记录的 ROUGE 分数
        try:
            scores = rouge.get_scores(ans, ground_truth, avg=True)
            rouge_1_scores.append(scores["rouge-1"]["f"])
            rouge_2_scores.append(scores["rouge-2"]["f"])
        except RecursionError:
            print(f"Skipping record due to RecursionError: {record}")

    # 计算 ROUGE-1 和 ROUGE-2 的平均分数
    avg_rouge_1 = sum(rouge_1_scores) / len(rouge_1_scores) if rouge_1_scores else 0
    avg_rouge_2 = sum(rouge_2_scores) / len(rouge_2_scores) if rouge_2_scores else 0

    return avg_rouge_1, avg_rouge_2


# 加载预测的摘要数据
with open("./abs_results.json", 'r') as f:
    predicted_summaries = json.load(f)

# 计算 ROUGE 分数
avg_rouge_1, avg_rouge_2 = evaluate_summary_rouge(predicted_summaries)
print("平均 ROUGE-1 分数:", avg_rouge_1)
print("平均 ROUGE-2 分数:", avg_rouge_2)


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Dumping model to file cache /tmp/jieba.cache
Dump cache file failed.
Traceback (most recent call last):
  File "/home/Data/.bin/.config/dz/anaconda3/envs/py_39/lib/python3.9/site-packages/jieba/__init__.py", line 154, in initialize
    _replace_file(fpath, cache_file)
PermissionError: [Errno 1] Operation not permitted: '/tmp/tmplg_3d6l2' -> '/tmp/jieba.cache'
Loading model cost 0.616 seconds.
Prefix dict has been built successfully.


平均 ROUGE-1 分数: 0.235464148037073
平均 ROUGE-2 分数: 0.07316036157490166


## 结果分析

本次任务 LLM 表现较差的可能原因是：
- LLM 生成的摘要与标准答案风格差异较大，导致分数偏低；
- prompt 仍可优化，可适当加入对生成摘要的风格、字数等要求的描述。